# **MUSIC GENERATION WITH AI**

**Install Required Libraries**

In [ ]:
!pip install tensorflow music21

**Import Libraries**

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from music21 import converter, instrument, note, chord, stream
import os
from collections import Counter

**Load and Preprocess the Data**

In [ ]:
def load_midi_file(file_path):
    notes = []
    if file_path.endswith(".mid"):
        midi = converter.parse(file_path)
        notes_to_parse = None
        try:
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse()
        except:
            notes_to_parse = midi.flat.notes
        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))
    return notes

# Load a single MIDI file
path = "/content/pakistan.mid"
notes = load_midi_file(path)
print(f"Loaded {len(notes)} notes from the MIDI file.")

# Prepare the sequence of notes
sequence_length = 100
pitchnames = sorted(set(item for item in notes))
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

network_input = []
network_output = []

for i in range(0, len(notes) - sequence_length, 1):
    sequence_in = notes[i:i + sequence_length]
    sequence_out = notes[i + sequence_length]
    network_input.append([note_to_int[char] for char in sequence_in])
    network_output.append(note_to_int[sequence_out])

n_patterns = len(network_input)
n_vocab = len(set(notes))

# Reshape the input into a format compatible with LSTM layers
network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
network_input = network_input / float(n_vocab)

network_output = tf.keras.utils.to_categorical(network_output)

Loaded 113 notes from the MIDI file.


**Build the Model**

In [ ]:
model = Sequential()
model.add(LSTM(512, input_shape=(network_input.shape[1], network_input.shape[2]), return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

**Train the Model**

In [ ]:
model.fit(network_input, network_output, epochs=200, batch_size=64)

Epoch 1/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - loss: 1.7574
Epoch 2/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step - loss: 1.7907
Epoch 3/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step - loss: 1.6944
Epoch 4/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step - loss: 1.8841
Epoch 5/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - loss: 1.7513
Epoch 6/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - loss: 1.7716
Epoch 7/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - loss: 1.8683
Epoch 8/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - loss: 1.8358
Epoch 9/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 1.7427
Epoch 10/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - loss: 1.8102
Epoch 11/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 1.8824
Epoch 12/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - loss: 1.8337
Epoch 13/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - loss: 1.8116
Epoch 14/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - loss: 1.7487
Epoch 15/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - loss: 1.7784
Epoch 16/200
1/1 

**Generate Music**

In [ ]:
def generate_notes(model, network_input, pitchnames, n_vocab):
    start = np.random.randint(0, len(network_input)-1)
    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
    pattern = network_input[start]
    prediction_output = []

    for note_index in range(500):
        prediction_input = np.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)
        prediction = model.predict(prediction_input, verbose=0)
        index = np.argmax(prediction)
        result = int_to_note[index]
        prediction_output.append(result)
        pattern = np.append(pattern, index)
        pattern = pattern[1:len(pattern)]
    return prediction_output

def create_midi(prediction_output):
    offset = 0
    output_notes = []
    for pattern in prediction_output:
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.quarterLength = 0.5
            output_notes.append(new_chord)
        else:
            new_note = note.Note(pattern)
            new_note.quarterLength = 0.5
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)
        offset += 0.5
    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='output.mid')

# Generate notes
prediction_output = generate_notes(model, network_input, pitchnames, n_vocab)
# Create MIDI file
create_midi(prediction_output)

**Download the Generated MIDI File**

In [ ]:
from google.colab import files
files.download('output.mid')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>